In [1]:
import copy
import time

import cv2
import torchvision.transforms as transforms

from utils.ImageList import ImageList
from utils.SerialObject import SerialObject
from utils.utils import channel12ToImgSize, getChannelValue, bgr8_to_jpeg

In [2]:
from jetcam.csi_camera import CSICamera
# from jetcam.usb_camera import USBCamera

camera = CSICamera(width=224, height=224)
# camera = USBCamera(width=224, height=224)
camera.running = True

In [3]:
project = "0502"
dataset = "test"
category = "local"

TRANSFORMS = transforms.Compose([
        transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),  # 改变图片的对比度 亮度 饱和度 色调
        transforms.Resize((224, 224)),  # 调整图片尺寸到[224, 224]
        transforms.ToTensor(),  # 转换到Tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        # 用平均值和标准差对浮点张量图像进行标准化 list内三个值对图片的三个通道使用不同值
    ])

In [4]:
global image_list, myserial
image_list = ImageList(project, category, dataset, transform=TRANSFORMS, random_hflip=True)
myserial = SerialObject(portNo="/dev/ttyACM0", baudRate=9600)
time.sleep(2)
print("当前接收机值: {}".format(getChannelValue(myserial, arduino_standardization=False, standardization=False)))

0502_test/local already creat
0502_test/local_labels already creat
0 files in the project folder
参数设置: 串口=/dev/ttyACM0, 波特率=9600
当前接收机值: ['0', '0', '0']


In [5]:
import ipywidgets
from IPython.display import display
import threading
import time

In [6]:
def draw_ruler_on_image(image,line_tinkness=1):
    changed_img = copy.copy(image)
    image_height = height
    image_weight = width
    helf_height = int(image_height/2)
    helf_weight = int(image_weight/2)
    # cv2.circle(image, (image_height / 2, image_weight / 2), 5, (255, 255, 255), 2)
    cv2.line(changed_img, (0, 0), (image_height, image_weight), (0, 0, 255), line_tinkness)
    cv2.line(changed_img, (0, image_height), (image_weight, 0), (0, 0, 255), line_tinkness)
    cv2.line(changed_img, (0, helf_weight), (image_weight, helf_weight), (0, 0, 255), line_tinkness)
    cv2.line(changed_img, (helf_height, 0), (helf_height, image_height), (0, 0, 255), line_tinkness)
    return changed_img

In [7]:

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ipywidgets.Image(width=camera.width, height=camera.height)
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
recor_button = ipywidgets.Button(description='记录')
show_button = ipywidgets.Button(description='显示视频(影响性能)')

# 当show_button被按下时 连接camera的value和camera_widget的value
def showCamera(c):        
    ipywidgets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
show_button.on_click(showCamera)

# ipywidgets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)


# create widgets
count_widget = ipywidgets.IntText(description='count', disabled=True, value=image_list.get_count())
global state
state = 1
# manually update counts at initialization
mem_img_list = []
def record(camera, count_widget, snapshot_widget, time_wait=0.2):
    global mem_img_list
    while True :
        if state != 1:
            start_time = time.time()
            
            channel_value = getChannelValue(myserial, 
                                            arduino_standardization=False, 
                                            standardization=True,
                                            max_value=2000, 
                                            min_value=1000, 
                                            CH1_bias=-8,
                                            CH2_bias=-16,
                                            CH3_bias=-12)
            image_value = camera.value
            mem_img_list.append([image_value, channel_value])
            count_widget.value = image_list.get_count() + len(mem_img_list)
            img_x, img_y = channel12ToImgSize(channel_value[0], channel_value[1])
            circle_img = image_list.draw_circle(image_value, int(img_x), int(img_y))
            snapshot_widget.value = bgr8_to_jpeg(circle_img)
            
            if len(mem_img_list) == 100:
                for single_data in mem_img_list:
                    image_list.save_image(single_data[0], single_data[1])
                mem_img_list = []
            
            finish_time = time.time()
            time_cost = finish_time - start_time
            if time_cost < time_wait:
                time.sleep(time_wait - float(time_cost))
        else:
            if mem_img_list != []:
                for single_data in mem_img_list:
                    image_list.save_image(single_data[0], single_data[1])
                mem_img_list = []
            time.sleep(0.5)
            


execute_thread = threading.Thread(target=record, args=(camera, count_widget, snapshot_widget))

def recordclick(c):
    global state
    print(state)
    print(recor_button.description)
    if recor_button.description == '记录':
        recor_button.description = '停止'
        state = 0
    else:
        recor_button.description = '记录'
        state = 1
recor_button.on_click(recordclick)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]),
    show_button,
    recor_button,
    count_widget
])

display(data_collection_widget)

execute_thread.start()

In [8]:
# !pwd

In [9]:
# ! zip -r 0422school_A.zip 0422school_A